In [1]:
import torch
import torch.nn as nn
from torch.nn import functional as F
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(device)
block_size = 8
batch_size = 4
max_iters = 1000
# eval_interval = 2500
learning_rate = 3e-4
eval_iters = 250

cpu


In [2]:
with open('wizard_of_oz.txt', 'r', encoding='utf-8') as f:
    text = f.read()
chars = sorted(set(text))
print(chars)
vocab_size = len(chars)


['\n', ' ', '!', '"', '&', "'", '(', ')', '*', ',', '-', '.', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', ':', ';', '?', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z', '[', ']', '_', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', '\ufeff']


In [3]:
string_to_int = { ch:i for i,ch in enumerate(chars) }
int_to_string = { i:ch for i,ch in enumerate(chars) }
encode = lambda s: [string_to_int[c] for c in s]
decode = lambda l: ''.join([int_to_string[i] for i in l])

data = torch.tensor(encode(text), dtype=torch.long)
# print(data[:100])


In [4]:
n = int(0.8*len(data))
train_data = data[:n]
val_data = data[n:]

def get_batch(split):
    data = train_data if split == 'train' else val_data
    ix = torch.randint(len(data) - block_size, (batch_size,))
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    x, y = x.to(device), y.to(device)
    return x, y

x, y = get_batch('train')
print('inputs:')
# print(x.shape)
print(x)
print('targets:')
print(y)



inputs:
tensor([[58,  9,  1, 76, 61, 58, 73, 61],
        [60, 61, 73,  1, 67, 62, 67, 58],
        [58,  0, 47, 62, 79, 54, 71, 57],
        [55, 68, 78,  9,  1, 60, 71, 54]])
targets:
tensor([[ 9,  1, 76, 61, 58, 73, 61, 58],
        [61, 73,  1, 67, 62, 67, 58,  1],
        [ 0, 47, 62, 79, 54, 71, 57, 11],
        [68, 78,  9,  1, 60, 71, 54, 75]])


In [5]:
@torch.no_grad()
def estimate_loss():
    out = {}
    model.eval()
    for split in ['train', 'val']:
        losses = torch.zeros(eval_iters)
        for k in range(eval_iters):
            X, Y = get_batch(split)
            logits, loss = model(X, Y)
            losses[k] = loss.item()
        out[split] = losses.mean()
    model.train()
    return out

In [6]:
class BigramLanguageModel(nn.Module):
    def __init__(self, vocab_size):
        super().__init__()
        self.token_embedding_table = nn.Embedding(vocab_size, vocab_size)
        
    def forward(self, index, targets=None):
        logits = self.token_embedding_table(index)
        
        
        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)
        
        return logits, loss
    
    def generate(self, index, max_new_tokens):
        # index is (B, T) array of indices in the current context
        for _ in range(max_new_tokens):
            # get the predictions
            logits, loss = self.forward(index)
            # focus only on the last time step
            logits = logits[:, -1, :] # becomes (B, C)
            # apply softmax to get probabilities
            probs = F.softmax(logits, dim=-1) # (B, C)
            # sample from the distribution
            index_next = torch.multinomial(probs, num_samples=1) # (B, 1)
            # append sampled index to the running sequence
            index = torch.cat((index, index_next), dim=1) # (B, T+1)
        return index

model = BigramLanguageModel(vocab_size)
m = model.to(device)

context = torch.zeros((1,1), dtype=torch.long, device=device)
generated_chars = decode(m.generate(context, max_new_tokens=500)[0].tolist())
print(generated_chars)



qZKE ;YOL'.Jf)xbE *DBi-*"lsyrfP9*k5kEk["2Tp[P[esnA .K0tG4NA8["p7u?[:V﻿EzS8i?6d5HBuq8uruG. LG9f1hqbq)37my0OPaIGBuU&:HdC"b3?JsW- v;OGiy(S.UfPC
U!-NLtLXyGMWOnfQ(vyG3e3a-?,:. EDv"EkW[l0EUSUl8XyQzo﻿gRAj﻿Ao(SbDNS8c_PasK0K1ZcEP9'mGYMTysiAO3Ks4EAWgj[Q2Mlc1OcYFBOGz9hp*Yic2p;204joW5xI]fiDNbk;yVL2AbwuK_P7?agP:Ul0Oc2;9D1z,;87u ur L(9vz)xlGsNAWdd0m8p;Z"E(7vtc[
CAg﻿o﻿ihWa .Do]2pbyjfne81g)3xd;tPtVqS8pw3z:.DxrlsDfoI,81dlb&tbGhlz ckI1 X_fz)V'W2Pt'kW';1zwRbfJL?-PdQHG4N(wZE!t9h9IVL(h)]J.DGEkMm.ELuqnc)82Xpa(yQU. G 


In [7]:
# create a PyTorch optimizer
optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)

for iter in range(max_iters):
    if iter % eval_iters == 0:
        losses = estimate_loss()
        print(f"step: {iter}, train loss: {losses['train']:.3f}, val loss: {losses['val']:.3f}")

    # sample a batch of data
    xb, yb = get_batch('train')

    # evaluate the loss
    logits, loss = model.forward(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()
print(loss.item())

step: 0, train loss: 4.977, val loss: 4.982
step: 250, train loss: 4.925, val loss: 4.898
step: 500, train loss: 4.853, val loss: 4.828
step: 750, train loss: 4.792, val loss: 4.786
4.691077709197998


In [8]:
context = torch.zeros((1,1), dtype=torch.long, device=device)
generated_chars = decode(m.generate(context, max_new_tokens=500)[0].tolist())
print(generated_chars)


(y0QOELGjXSr[AJmlb'(k(j8kS[,GE'yK V9FyTTD3p. V*eHTjWigOCZGbF5Tf)x ",o[A7uBOGPtAHzI'"ba lrbYWr4vz)LBumpjwne[1a XyRgPcJFyi'z,he "Cy﻿oM0M!(PzXLdIwCBuBuh&T1xOHOjVYc:i:WInagE)vQI1tHdPy6t4x!]fTzlkW[7qBil*-xF?69[CnbMY'WEEO8QzNAgnz1ZFb'ZPoi.zQ
p-la-CakynZJ]Tv
[9, bY4j9L"PPAgX!CKp2JB-y
S)*Dw"GcR)wT!py6QExdd;3-ZrWp)vg-Rb![v4[1da
3, PPtS&ai"P]BDz,GZhtpaOHQyqkkM?A;2AWcCY:WEDC:jRv1kC﻿yT﻿6Ufaht
wvo*]f9Hd;HJYO:X05JQVqT??X0m8﻿5GJVbY0y6bI)jEOa&MG?JK GtpR1Q1TN7"s)jc)'mGLO65A;c:UlLOQH?4L9Vtp0AvQ56axdV3oIJ[iBDG]XLV
